# 1. Staircase Sweep Measurements, I/V
## 1.1. Notes on examples from *Programming Guide pp-198*

**Table1.** Staircase sweep measurements commands

| Function | Command | Parameters | PymeasureMethod
| --- | --- | --- | --- |
| Enables channels | **CN** | ```[chnum ... [,chnum] ... ]``` | ```SMU.enable()``` |
| Disable channels | **CL** | ```[chnum ... [,chnum] ... ]``` | ```SMU.disable()``` |
| Set Filters ON/OFF | **```[FL]```** | ```mode[,chnum ... [,chnum] ... ``` | ```SMU.filter()``` |
| Set series resistor ON/OFF | **```[SSR]```** | ```chnum,mode ``` | ```series_resistor``` |
| Set integration Time | **```AV```** | ```number[,mode] ``` | ```adc_averaging()``` |
| Set integration Time | **```AAD```** | ```chnum[,type] ``` | ```SMU.adc_type()``` |
| Set integration Time | **```AIT```** | ```mode,mode[,N] ``` | ```adc_setup()``` |
| Sets sweep source timing parameter | **```[WT]```** | ```hold,delay [,sdelay[,tdelay[,mdelay]]] ``` | ```sweep_timing()``` |
| Sets auto abort function | **```[WM]```** | ```abort[,post] ``` | ```sweep_auto_abort()``` |
| Set Voltage Sweep Source | **WV** | ```chnum,mode,range,start,stop, step[,comp[,Pcomp]]``` | ```SMU.staircase_sweep_source() mode: 'VOLTAGE'``` |
| Set Current Sweep Source | **WI** | ```chnum,mode,range,start,stop, step[,comp[,Pcomp]]``` | ```SMU.staircase_sweep_source() mode: 'CURRENT'``` |
| Set synchronous Sweep Source Voltage | **```[WSV]```** | ```chnum,range,start,stop [,comp[,Pcomp]] ``` | ```SMU.synchronous_sweep_source() mode: 'VOLTAGE'``` |
| Set synchronous Sweep Source current | **```[WSI]```** | ```chnum,range,start,stop [,comp[,Pcomp]]``` | ```SMU.synchronous_sweep_source() mode: 'CURRENT'``` |
| Forces constant voltage | **```DV, TDV```** | ```chnum,range,output [,comp[,polarity[,crange]]] ``` | ```SMU.force() mode: 'VOLTAGE'``` |
| Forces constant current | **```DI, TDI```** | ```chnum,range,output [,comp[,polarity[,crange]]] ``` | ```SMU.force() mode: 'CURRENT'``` |
| Sets voltage measurement range | **```[RV]```** | ```chnum,range ``` | ```meas_range_voltage``` |
| Sets current measurement range | **```[RI]```** | ```chnum,range ``` | ```meas_range_current``` |
| Sets current measurement range | **```[RM]```** | ```chnum,mode[,rate] ``` | ```SMU.meas_range_current_auto()``` |
| Sets measurement mode | **```MM```** | ```2,chnum[,chnum ... [,chnum] ... ]``` | ```meas_mode()``` |
| Sets SMU operation mode | **```[CMM]```** | ```chnum,mode ``` | ```SMU.meas_op_mode()``` |
| Execute Measurement | **```XE```** | ```N/A ``` | ```send_trigger()``` |


**Note**:
* **a**. The ```WSV/WSI``` command must be entered after the ```WV/WI``` command.

**Usage of commands not listed in Table1**

| Function CMD | Command in this example | Parameters | Explaination 
| --- | --- | --- | --- |
| ```FMT``` | ```FMT 5,1``` | ```FMT format[,mode]``` | **Mode** stands for data output mode, i.e, source data returned with measurement data|
| ```TSC``` | ```TSC 1``` | ```0 or 1``` | Disable/enable timestamp function|
| ```ERR``` | ```ERR? 1``` | ```0 or 1``` | **0:** Reads the content from the error queue and **clears** the queue, returns for error codes in the order of their occurrence  <br/> **1:** Reads from the head of error queue and removes the code from the queue. This return one error code|

## 1.2. Convert Pymeasure-stock to fit b1505a

**Stock B1500 base class structure is**:
* L46 - L912: Skeleton class of B1500
* L918 - L1362: SMU class
* L1368 - L1572: Ranging classes

* L1575 - L1696: Helpler classes for parameter capturing
* L1703 - End: Query Learn class: parse instrument setting to human readable format

### 1.2.1. Code segments that we don't want to mess
We are okay with most of the methods provided in the stock-version pymeasure b1500 class:
* L167 to L232 should be compatiable with b1505a
* L238 to L508 are about data formatting, no intention to mess with these for now (05/07)


**EXCEPT FOR THE FOLLOWING:**

### 1.2.2. Problems with stock-version Pymeasure 
#### a. SMU setups
B1505a SMU module query differs from B1500:
```python
    module_names_b1500 = {
        'B1525A': 'SPGU',
        'B1517A': 'HRSMU',
        'B1511A': 'MPSMU',
        'B1511B': 'MPSMU',
        'B1510A': 'HPSMU',
        'B1514A': 'MCSMU',
        'B1520A': 'MFCMU'
    }
```

RER b1505a involves the following SMUs
```python
    0:('GNDU','GNDU+ADC') # ? Might not be a real SMU
    1:('HPSMU','SMU1:HP') # B1510A 
    3:('HPSMU','SMU2:HP') # B1511B ? B1505A only have one HPSMU, I suppose it should be MPSMU
    5:('MFCMU','CMU1:MF') # B1520A
    6:('HCSMU','SMU3:HC') # B1512A
    8:('HVSMU','SMU4:HV') # B1513A
```

That is, we may need to override ```b1500.query_modules()``` method because:
* B1500 is equipped with ```SPGU and HRSMU```, but b1505a is NOT
* RER B1505a uses ```HVSMU(8)``` and ```HCSMU(6)```, BUT b1500 NOT equipped with these SMUs

#### b. ADC setups
ADC



**NOTE:** Legacy RER b1505a code could be inaccurate regarding the definitions of SMUs, though this issue might not affect the usage